In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import pi, cos, sin, sqrt, atan2
import random
from os.path import exists
from rtsvg import *
rt = RACETrack()
# Load the data
edges_filename  = '../../data/stanford/facebook/1912.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
my_df             = pl.DataFrame(_lu_)

_to_remove_ = set([428, 563, 1967]) # for 1912 graph
my_df = my_df.filter(~pl.col('fm').is_in(_to_remove_) & ~pl.col('to').is_in(_to_remove_))

g                 = rt.createNetworkXGraph(my_df, [('fm','to')])
degree_sorter     = []
for _node_ in g.nodes(): degree_sorter.append((g.degree[_node_], _node_))
degree_sorter.sort(reverse=True)
high_degree_nodes = set([_node_[1] for _node_ in degree_sorter[:10]])

In [8]:
# Find the communities
df_minus_high_degree_nodes = my_df.filter(~pl.col('fm').is_in(high_degree_nodes) & ~pl.col('to').is_in(high_degree_nodes))
g_minus_high_degree_nodes  = rt.createNetworkXGraph(df_minus_high_degree_nodes, [('fm','to')])
communities       = nx.community.louvain_communities(g_minus_high_degree_nodes)
community_lu      = {}
cluster_number    = max(max(my_df['fm']),max(my_df['to'])) + 1000 # needs to be different if nodes are strings
for _node_ in high_degree_nodes: 
    community_lu[cluster_number] = set([_node_])
    cluster_number += 1
for _community_ in communities:
    if len(_community_) <= 8: # if the community it too small, pull those nodes out
        for _node_ in _community_:
            community_lu[cluster_number] = set([_node_])
            cluster_number += 1
    else:
        community_lu[cluster_number] = set(_community_)
        cluster_number += 1

In [9]:
# Collapse the communities
my_df_communities = rt.collapseDataFrameGraphByClusters(my_df, [('fm','to')], community_lu)
g_communities     = rt.createNetworkXGraph(my_df_communities, [('__fm__','__to__')])
communities_pos   = nx.spring_layout(g_communities)

# Find the bounding box and determine the coordinate transforms
x_min, y_min, x_max, y_max = rt.positionExtents(communities_pos)
w, h         = 900, 900
cd_r         = 64
x_ins, y_ins = cd_r + 10, cd_r + 10
wxToSx       = lambda x: 1.5*cd_r + 5 + (w - 3*cd_r)*(x-x_min)/(x_max-x_min)
wyToSy       = lambda y: 1.5*cd_r + 5 + (h - 3*cd_r)*(y-y_min)/(y_max-y_min)

# Crunch the circles
communities_keys_ordered = sorted(list(community_lu.keys()))
circles                  = []
for i in range(len(communities_keys_ordered)):
    _community_key_ = communities_keys_ordered[i]
    sx, sy = wxToSx(communities_pos[_community_key_][0]), wyToSy(communities_pos[_community_key_][1])
    if len(community_lu[_community_key_]) == 1: circles.append((sx, sy, 20.0))
    else:                                       circles.append((sx, sy, cd_r))

circles_adjusted  = rt.crunchCircles(circles)
circles_finalized = []

# And re-assign back into the pos array -- these again need to be normalized into the screen view
for_extent_calculation = {}
for i in range(len(circles_adjusted)): for_extent_calculation[i] = circles_adjusted[i][0], circles_adjusted[i][1]
x_min, y_min, x_max, y_max = rt.positionExtents(for_extent_calculation)
wxToSx       = lambda x: 1.5*cd_r + 5 + (w - 3*cd_r)*(x-x_min)/(x_max-x_min)
wyToSy       = lambda y: 1.5*cd_r + 5 + (h - 3*cd_r)*(y-y_min)/(y_max-y_min)
_pts_        = []
for i in range(len(communities_keys_ordered)):
    _community_key_ = communities_keys_ordered[i]
    sx,sy = wxToSx(circles_adjusted[i][0]), wyToSy(circles_adjusted[i][1])
    circles_finalized.append((sx, sy, circles_adjusted[i][2]))
    communities_pos[_community_key_] = (sx,sy)
    _pts_.append((sx,sy))

# Voronoi
voronoi_cells = rt.isedgarVoronoi(_pts_, Box=[(0,h),(w,h),(w,0),(0,0)])

In [ ]:
svg = [f'<svg x="0" y="0" width="{w}" height="{h}"><rect x="0" y="0" width="{w}" height="{h}" fill="white" />']

# Render the communities as chord diagrams
dfs_rendered, _node_to_cd_, _cd_sxy_ = [], {}, {}
for _community_ in community_lu:
    _df_   = my_df.filter(pl.col('fm').is_in(community_lu[_community_]) & pl.col('to').is_in(community_lu[_community_]))
    dfs_rendered.append(_df_)
    if len(community_lu[_community_]) == 1:
        # sx, sy = wxToSx(communities_pos[_community_][0]), wyToSy(communities_pos[_community_][1])
        sx, sy = communities_pos[_community_][0], communities_pos[_community_][1]
        _node_ = list(community_lu[_community_])[0]
        _cd_sxy_[_node_] = (sx,sy)
        svg.append(f'<circle cx="{sx}" cy="{sy}" r="4" stroke="black" stroke-width="1.5" fill="black" />')
    else:
        _opacity_ = max(0.8 - len(community_lu[_community_])/20.0, 0.02)
        _cd_   = rt.chordDiagram(_df_, [('fm', 'to')], w=cd_r*2, h=cd_r*2, x_ins=0, y_ins=0, link_opacity=_opacity_, draw_border=False, node_h=4)
        for _node_ in community_lu[_community_]: _node_to_cd_[_node_] = _cd_
        # sx, sy = wxToSx(communities_pos[_community_][0]), wyToSy(communities_pos[_community_][1])
        sx, sy = communities_pos[_community_][0], communities_pos[_community_][1]
        _cd_sxy_[_cd_] = (sx,sy)
        svg.append(f'<g transform="translate({sx-cd_r}, {sy-cd_r})">{_cd_._repr_svg_()}</g>')

# Determine the edges between the communities and render them
df_rendered    = pl.concat(dfs_rendered)
df_remaining   = my_df.join(df_rendered, on=['fm', 'to'], how='anti')
for k, k_df in df_remaining.group_by(('fm', 'to')):
    if k[0] in _node_to_cd_:
        _cd_fm_            = _node_to_cd_[k[0]]
        fm_cd_sx, fm_cd_sy = _cd_sxy_[_cd_fm_]
        fm_eps             = _cd_fm_.entityPositions(k[0]) 
        fm_x, fm_y         = fm_eps[0].xy()
        fm_cd_r            = cd_r
    else:
        fm_x, fm_y                    =  _cd_sxy_[k[0]]
        fm_cd_r = fm_cd_sx = fm_cd_sy = 0.0

    if k[1] in _node_to_cd_:
        _cd_to_            = _node_to_cd_[k[1]]    
        to_cd_sx, to_cd_sy = _cd_sxy_[_cd_to_]
        to_eps             = _cd_to_.entityPositions(k[1])
        to_x, to_y         = to_eps[0].xy()
        to_cd_r            = cd_r
    else:
        to_x, to_y                    = _cd_sxy_[k[1]]
        to_cd_r = to_cd_sx = to_cd_sy = 0.0
    svg.append(f'<line x1="{fm_x+fm_cd_sx-fm_cd_r}" y1="{fm_y+fm_cd_sy-fm_cd_r}" x2="{to_x+to_cd_sx-to_cd_r}" y2="{to_y+to_cd_sy-to_cd_r}" stroke="#ff0000" stroke-opacity="0.5" stroke-width="0.1" />')

# Render the cells -- shrinking them when necessary
min_from_r = 20.0
first_cell = True
for cell_i in range(len(voronoi_cells)):
    _cell_ = voronoi_cells[cell_i]
    _path_ = f'M {_cell_[0][0]} {_cell_[0][1]} '
    for i in range(1, len(_cell_)): _path_ += f'L {_cell_[i][0]} {_cell_[i][1]} '
    _path_ += 'z'
    svg.append(f'<path d="{_path_}" stroke="black" stroke-opacity="0.5" stroke-width="4.0" fill="black" fill-opacity="0.1" />')

svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
rt.link(my_df_communities, [('__fm__','__to__')], communities_pos, draw_labels=True, w=800, h=800)

In [ ]:
df_test  = pl.DataFrame({'fm':"a b c d".split(),
                         'to':"b c d a".split(),
                         'ct':[99.0,0.1,99.0,0.1]})
g_test   = rt.createNetworkXGraph(df_test, [('fm','to')], count_by='ct')
pos_test = nx.spring_layout(g_test)
rt.link(df_test, [('fm','to')], pos_test, draw_labels=True, w=800, h=800)